# tab4 注解

## 所需库

In [1]:
from PyQt5 import QtWidgets,QtCore, QtGui
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPicture,QPainter
from PyQt5.QtCore import QPointF,QRectF
import sys
import zmq
import pyqtgraph as pg
import time
from datetime import datetime
import pandas as pd
import numpy as np
import random
import configparser
import os

## 所需要的类

In [2]:
class DrawRecItem(pg.GraphicsObject):
    def __init__(self,data):
        super().__init__()
        self.data=data
        self.color_list=[(217, 194, 234),
                         (58, 135, 162),
                         (196, 62, 244),
                         (147, 223, 153),
                         (103, 182, 144),
                         (194, 22, 47),
                         (22, 182, 89)]
        self.draw_rect()
        
        
    def draw_rect(self):
        
        self.picture=QPicture()
        p1=QPainter(self.picture)
        
        ### 画k线图 ##########
        
        # 设置画pen 颜色，用来画线
        # 颜色代码使用 RGB值，缩写，都可以
#         p1.setPen(pg.mkPen((0,0,0)))
#         for i in range(len(self.data)):
#             #画一条最大值最小值之间的线
#             p1.drawLine(QPointF(i,self.data[i][3]),QPointF(i,self.data[i][2]))
#             # 设置画刷颜色
#             if self.data[i][1]>self.data[i][4]:
#                 p1.setBrush(pg.mkBrush('g'))
#             else:
#                 p1.setBrush(pg.mkBrush('r'))
#             p1.drawRect(QRectF(i-0.3,self.data[i][1],0.6,self.data[i][4]-self.data[i][1]))
            
        #### 画自定义的线 ####################
#          ## TEST close线
#         p1.setPen(pg.mkPen(255,0,0))
#         for i in range(len(self.data['TEST'])-1):
#             p1.drawLine(QPointF(i,self.data['TEST'][i][4]),QPointF(i+1,self.data['TEST'][i+1][4]))
        
        count=0
        for index in self.data.keys():
            rand_col=self.color_list[count]
            p1.setPen(pg.mkPen(rand_col))
            for i in range(len(self.data[index])-1):
                p1.drawLine(QPointF(i,self.data[index][i][4]),QPointF(i+1,self.data[index][i+1][4]))
            count+=1
#         ## FLOW  close线
#         p1.setPen(pg.mkPen(255,0,0))
#         for i in range(len(self.data['FLOW'])-1):
#             p1.drawLine(QPointF(i,self.data['FLOW'][i][4]),QPointF(i+1,self.data['FLOW'][i+1][4]))
        
#         ## FLML  close线
#         p1.setPen(pg.mkPen(0,255,0))
#         for i in range(len(self.data['FLML'])-1):
#             p1.drawLine(QPointF(i,self.data['FLML'][i][4]),QPointF(i+1,self.data['FLML'][i+1][4]))
            
#         ## ML  close线
#         p1.setPen(pg.mkPen(0,0,255))
#         for i in range(len(self.data['ML'])-1):
#             p1.drawLine(QPointF(i,self.data['ML'][i][4]),QPointF(i+1,self.data['ML'][i+1][4]))
        
    def paint(self,p,*args):
        p.drawPicture(0,0,self.picture)
    def boundingRect(self):
        return QRectF(self.picture.boundingRect())


## 时间轴类

In [3]:
class MyAxisItem(pg.AxisItem):
    def __init__(self,ticks,*args,**kwargs):
        pg.AxisItem.__init__(self,*args,**kwargs)
        self.x_values=[x[0] for x in ticks]
        self.x_strings=[x[1] for x in ticks]
    
    def tickStrings(self, values, scale, spacing):
        strings=[]
        for v in values:
            vs=v*scale
            if vs in self.x_values:
                vstr=self.x_strings[np.abs(self.x_values-vs).argmin()]

            else:
                vstr=''
            
            strings.append(vstr)
        return strings

## tab4 线程

In [4]:
#### tab4 线程############################
class Update_tab4(QtCore.QThread):
    requestChanged = QtCore.pyqtSignal(tuple)

    def __init__(self, parent=None):
        super(Update_tab4, self).__init__(parent)
        # 设置工作状态与初始num数值


    def __del__(self):
        # 线程状态改变与线程终止
        self.working = False
        self.wait()

    def run(self):
        context = zmq.Context()
        sock = context.socket(zmq.SUB)
        sock.setsockopt(zmq.SUBSCRIBE,b'BarraRisk')
        
        sock.setsockopt(zmq.HEARTBEAT_IVL,     5000)
        sock.setsockopt(zmq.HEARTBEAT_TIMEOUT, 3000)

        sock.connect('tcp://192.168.1.108:19006')
#         for i in range(100):
        while True:
#             msg=get_stock_info()
#             time.sleep(1)
            sss=sock.recv()
            msg = sss.decode("ascii").replace(' ','').split(",")
            new_data_collection=[]
                
#             if msg[0]=='ML':
#                 print(msg)
            
            lag=msg[1]
            new_data_collection.append(msg[1])
            new_data_collection.append(msg[2])
            new_data_collection.append(float(msg[3]))
            self.requestChanged.emit(tuple(new_data_collection))
            
                

## 主界面

In [5]:
## 主基类，是 整个GUI的主窗口，内部含有三个子窗口
class Control_sys_Tab(QTabWidget):
    def __init__(self, parent=None):
        
        self.Data={
                  }
        self.color_list = [(193, 210, 240),
                           (58, 135, 162),
                           (196, 62, 244),
                           (147, 223, 153),
                           (103, 182, 144),
                           (194, 22, 47),
                           (22, 182, 89)]
        
        
        
        self.time_dict={}
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        
        # 判断有无历史数据
        self.with_hist=False
        
        self.g_CurrErrorRow  = 0
        super().__init__(parent)
        
#         # 设置sizepolicy
#         self.setSizePolicy(QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.MinimumExpanding,
#                                                  QtWidgets.QSizePolicy.MinimumExpanding))
        
        self.setObjectName("Control_system")
        self.resize(1800, 985)
        self.setWindowTitle("实时监控系统")

        # 创建4个选项卡小控件窗口
        self.tab1 = QWidget()
        self.tab2 = QWidget()
        self.tab3 = QWidget()
        self.tab4 = QWidget()

        # 将四个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "损益界面")
        self.addTab(self.tab3, "风险分析")
        self.addTab(self.tab4, "Barra分析")

        # 记录tab2 中的数据，后续会从其他类中读取数据，并更新和作图
        self.conf=configparser.ConfigParser()
        self.conf.read('./depend/config.txt')
        
        # tab4 use
        # 对于Barra数据，现计划是不断刷新十行展示数据，那么只需要使用一维map来存储即可 map的key值为策略名，value为顺序的list
        self.Barra_Data={}
        config_tab4=self.conf.get('tab4','table7_lables').split(',')
        for i in config_tab4:
            self.Barra_Data[i]={}
        self.tab4_work=Update_tab4()
        
        # risk 列表
        self.risk_list=['BETA',"BP","EP","GRO","LEV","LIQ","MOM","NLSIZE","SIZE","VOL"]

        # 每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()
        self.tab4UI()

    def tab1UI(self):
        
        
        return    

    def tab2UI(self):
        
        
        return
        
    def tab3UI(self):

        return
    
    def tab4UI(self):
        
        # 设置布局
        # 由于只需要两个窗口，因此一个布局器即可
        layout=QVBoxLayout()
        
        self.tableWidget7=QtWidgets.QTableWidget()
        self.tableWidget7.setRowCount(10)
        self.tab4_lable_list=self.conf.get('tab4','table7_lables').split(',')
        self.tableWidget7.setColumnCount(len(self.tab4_lable_list))
        self.tableWidget7.setObjectName("tableWidget7")
        
        self.tableWidget7.setHorizontalHeaderLabels(self.tab4_lable_list)
        
        
        # 设置左侧表单的长宽高
        for i in range (0,10):
            self.tableWidget7.setRowHeight(i,30)
        
        for i in range(0,len(self.tab4_lable_list)):
            self.tableWidget7.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.tableWidget7.setColumnWidth(i, 300)
        
        ################## 右布局 #################################      
        ## 首先要生成对应的坐标轴类
        temp_list=np.arange(len(self.risk_list))
        ticks = [(i, j) for i, j in zip(temp_list, self.risk_list)]
        strAxis = MyAxisItem(ticks, orientation="bottom")
        
        
        # 绘制柱形图所所在位子信息
        self.plotWidget7=pg.PlotWidget(axisItems={'bottom': strAxis})
        self.plotWidget7.addLegend()
        self.plotWidget7.setFixedHeight(600)
        ## 设置背景色
        self.plotWidget7.setBackground((255, 255, 255))
        
        #### 测试用 绘图###############################################
#         # create list of floats
#         y1 = np.linspace(0, 20, num=20)
        
#         # create horizontal list
#         x1 = np.arange(20)

#         # create bar chart
#         bg1 = pg.BarGraphItem(x=x1, height=y1, width=0.6, brush='r')
        
#         self.plotWidget7.addItem(bg1)
        ###############################################################
        
        # 添加控件进入布局
        layout.addWidget(self.tableWidget7)
        layout.addWidget(self.plotWidget7)
        self.tab4.setLayout(layout)
        
        # 直接启动进程
        self.slotStart_tab4()
        
############################################### tab4 双击事件 ################################################################
        self.tableWidget7.itemDoubleClicked.connect(self.tab4_clicked)
 
    def tab4_clicked(self,event=None):
        #获得被点击的对象
        try:
            obj=event
            col_index=self.tab4_lable_list[obj.column()]
            ######################################
            #测试用
            print(self.Barra_Data[col_index])
            ######################################


            # 当数据完整，即每一个策略对应的 风险数据都有时，开始画图
            if len(self.Barra_Data[col_index])>=10:
                # 首先需要按顺序生成 对应的列表
                temp_list=[]
                for i in self.risk_list:
                    temp_list.append(self.Barra_Data[col_index][i])
                # create list of floats
                y1 = temp_list[:]

                # create horizontal list
                x1 = np.arange(len(y1))

                # create bar chart
                bg1 = pg.BarGraphItem(x=x1, height=y1, width=0.6, brush=(144, 238, 144),name=col_index+' Risk')

                self.plotWidget7.clear()
                self.plotWidget7.addItem(bg1)
        except Exception as e:
            print("error in tab4_clicked")
        
##############################################tab4 slot#################################################
    @QtCore.pyqtSlot()
    def slotStart_tab4(self):
        # 开启一个新进程用来 更新数据
        self.update_data_thread4 = Update_tab4(self) 
        self.update_data_thread4.requestChanged.connect(self.onRequestChanged_tab4)
        # 线程进入 准备阶段
        self.update_data_thread4.start()

    @QtCore.pyqtSlot(tuple)
    def onRequestChanged_tab4(self,text):
        # text 即为我们所需要的数据列
        # print(text)
        
        
        lable_list=self.conf.get('tab4','table7_lables').split(',')
        
        # 设置第一列
        for i,element in enumerate(self.risk_list):
            it = QtWidgets.QTableWidgetItem()
            self.tableWidget7.setItem(i,0,it)
            it.setTextAlignment(Qt.AlignCenter)
            it.setText(element)
            
        # 根据 传过来的数据设置对应格子
        
        col_id=self.tab4_lable_list.index(text[1])
        row_id=self.risk_list.index(text[0])
        it = self.tableWidget7.item(row_id,col_id)
        if it is None:
            it = QtWidgets.QTableWidgetItem()
            self.tableWidget7.setItem(row_id, col_id, it)
        self.Barra_Data[text[1]][text[0]]=text[2]
        it.setText(str(text[2]))
        it.setTextAlignment(Qt.AlignCenter)
        

# Main

In [6]:
if __name__ == "__main__":
    app = QtWidgets.QApplication.instance()
    if app is None:
         app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())

{'SIZE': -0.2068, 'VOL': 0.0502, 'BETA': 0.1343, 'BP': -0.1886, 'EP': -0.0565, 'GRO': -0.0245, 'LEV': -0.0981, 'LIQ': 0.1827, 'MOM': 0.0511, 'NLSIZE': 0.1935}
{'NLSIZE': 0.9563, 'SIZE': -0.0636, 'VOL': -0.1375, 'BETA': 0.0441, 'BP': -0.4184, 'EP': -0.0594, 'GRO': 0.1108, 'LEV': -0.3013, 'LIQ': -0.1457, 'MOM': 0.2568}
{'NLSIZE': 0.1893, 'SIZE': -0.211, 'VOL': -0.0353, 'BETA': -0.0801, 'BP': -0.1855, 'EP': -0.0264, 'GRO': -0.0483, 'LEV': -0.157, 'LIQ': -0.066, 'MOM': -0.0276}
{'SIZE': -0.2068, 'VOL': 0.0502, 'BETA': 0.1343, 'BP': -0.1886, 'EP': -0.0565, 'GRO': -0.0245, 'LEV': -0.0981, 'LIQ': 0.1827, 'MOM': 0.0511, 'NLSIZE': 0.1935}
{'NLSIZE': 0.1893, 'SIZE': -0.211, 'VOL': -0.0353, 'BETA': -0.0801, 'BP': -0.1855, 'EP': -0.0264, 'GRO': -0.0483, 'LEV': -0.157, 'LIQ': -0.066, 'MOM': -0.0276}
{'NLSIZE': 0.9563, 'SIZE': -0.0636, 'VOL': -0.1375, 'BETA': 0.0441, 'BP': -0.4184, 'EP': -0.0594, 'GRO': 0.1108, 'LEV': -0.3013, 'LIQ': -0.1457, 'MOM': 0.2568}


SystemExit: 0

C:\Users\taosu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
